- Prepare input for FitHiC2
- Run FitHiC2
- Filter results

In [1]:
import pandas as pd
import numpy as np
import cooler
import bioframe as bf
import utils

# Input data to run this notebook

In [2]:
"""In this notebook we use our Hi-C data combined with Hi-C 
from Hu et al., 2021: https://doi.org/10.1038/s41467-021-24243-0
Thus, to reproduce figures from the manuscript, one has to create
mcool files from Hu2021 raw data and then merge it with mcools
from this study.

Additionally, this notebook can be applied to merged snm3C-seq data from  
Tian et al., 2023: https://doi.org/10.1126/science.adf5357
Tian2023 data includes merged data for excitatory (EN) and inhibitory 
neurons (IN) of MTG.
"""

# Path to cooler maps
# Comment out cell types that are not needed
clr_path = {
    'NeuN+': "../../NeuNplus.our_and_Hu2021_merged.mcool",
    'NeuN-': "../../NeuNminus.our_and_Hu2021_merged.mcool",
    'EN': "MTG.EN.2786_cells.10kb.mcool",
    'IN': "MTG.IN.2786_cells.mcool"
}

# Path to neuronal dot anchors
neuronal_dot_anchor_path = "../data/Suppl. Table S2. Neuronal dot anchors.xlsx"

# Create input for fithic

In [ ]:
# This section processes one cell type at a time
# Select cell type
ct = 'NeuN+'

In [5]:
# Load hi-c matrices
res = 100_000
clr = cooler.Cooler(clr_path[ct] + f'::resolutions/{res}')

In [8]:
# Get bias values
bins = clr.bins()[:]
bins = bins.loc[bins['chrom'] != 'chrM']
bins['midpoint'] = ( (bins['start'] + bins['end']) / 2 ).astype(int)

# From fithic paper: "As long as the bias values are scaled to have an average of 1 
# and high values represent loci with higher overall raw counts, 
# FitHiC2 will be able to use them in significance assignment."
bins['bias'] = (1 / bins['weight']) / (1 / bins['weight']).mean()
bins.head()

,chrom,start,end,weight,midpoint,bias
0,chr1,0,100000,NaN,50000,NaN
1,chr1,100000,200000,NaN,150000,NaN
2,chr1,200000,300000,NaN,250000,NaN
3,chr1,300000,400000,NaN,350000,NaN
4,chr1,400000,500000,NaN,450000,NaN


In [9]:
# Save bias values to file
out_name = f'fithic.bias.{ct}.{res // 1000}kb.txt.gz'

bins[['chrom', 'midpoint', 'bias']]\
    .to_csv(out_name, sep='\t', header=False, index=False, na_rep=-1)

In [10]:
# Get fragments values
fr_out_name = f'fithic.fragments.{ct}.{res // 1000}kb.txt.gz'

mtx_arr = np.zeros(bins.shape[0])

# Get column sums of cis contacts
k=0
for i, chrom1 in enumerate(clr.chromnames):
    for j, chrom2 in enumerate(clr.chromnames):
        if i <= j:
            mtx = clr.matrix(balance=False).fetch(chrom1, chrom2)
            if i == j:
                mtx[np.arange(mtx.shape[0]), np.arange(mtx.shape[0])] = 0 # del 1st diag
            mtx_colsum = np.sum(mtx, axis=1)
            mtx_arr[k:k+len(mtx_colsum)] += mtx_colsum
            
    k+=len(mtx_colsum)
    
bins['colsum'] = mtx_arr.astype(int)

# Save fragments values to a file
bins[['chrom', 'start', 'midpoint', 'colsum', 'end']]\
    .to_csv(fr_out_name, sep='\t', header=False, index=False)

In [11]:
# Get and save interaction values
in_out_name = f'fithic.interactions.{ct}.{res // 1000}kb.txt.gz'

for i, chrom in enumerate(clr.chromnames):
    pix = clr.pixels(join=True).fetch(chrom)
    pix = pix.loc[pix['count'] > 0]
    pix.loc[:, 'count'] = pix['count'].astype(int)
    pix['midpoint1'] = ( (pix['start1'] + pix['end1']) / 2 ).astype(int)
    pix['midpoint2'] = ( (pix['start2'] + pix['end2']) / 2 ).astype(int)
    if i == 0:
        pix[['chrom1', 'midpoint1', 'chrom2', 'midpoint2', 'count']]\
            .to_csv(in_out_name, sep='\t', header=False, index=False)
    else: # append
        pix[['chrom1', 'midpoint1', 'chrom2', 'midpoint2', 'count']]\
            .to_csv(in_out_name, mode='a', sep='\t', header=False, index=False)

pd.read_table(in_out_name, nrows=10).head()

,chr1,50000,chr1.1,50000.1,41
0,chr1,50000,chr1,150000,306
1,chr1,50000,chr1,250000,28
2,chr1,50000,chr1,350000,1
3,chr1,50000,chr1,650000,9
4,chr1,50000,chr1,850000,1


# Run fithic from console

# Filter fithic results to keep significant interactions

In [6]:
# Load fithic table and drop non-significant interactions
# fithic_path = {
#     'NeuN+': 'fithic.NeuN+.100kb.all/FitHiC.spline_pass1.res100000.significances.txt.gz',
#     'NeuN-': 'fithic.NeuN-.100kb.all/FitHiC.spline_pass1.res100000.significances.txt.gz',
#     'EN':    'fithic.EN.100kb.all/FitHiC.spline_pass1.res100000.significances.txt.gz',
#     'IN':    'fithic.IN.100kb.all/FitHiC.spline_pass1.res100000.significances.txt.gz'
# }
fithic_path = {
    'NeuN+': '../../../2022.10.07/fithic.100kb.NeuN+.ourAndHu2021_merged.sampled.all/FitHiC.spline_pass1.res100000.significances.txt.gz',
    'NeuN-': '../../../2022.10.07/fithic.100kb.NeuN-.ourAndHu2021_merged.sampled.all/FitHiC.spline_pass1.res100000.significances.txt.gz',
    'EN': '../../fithic.EN.MTG.Tian2023.sampled.100kb.all/FitHiC.spline_pass1.res100000.significances.txt.gz',
    'IN': '../../fithic.IN.MTG.Tian2023.sampled.100kb.all/FitHiC.spline_pass1.res100000.significances.txt.gz'
}
cts = list(fithic_path.keys())

fithic = {
    ct: pd.read_table( 
            path, 
            usecols=['chr1', 'fragmentMid1', 'chr2', 'fragmentMid2', 'q-value'], 
            dtype={'chr1': 'category', 'fragmentMid1': int, 'chr2': 'category', 'fragmentMid2': int},
        )
    for ct, path in fithic_path.items()
}

for ct in cts:
    print(ct)
    fithic[ct] = fithic[ct].loc[fithic[ct]['q-value'] < 0.05].reset_index(drop=True)\
                           .rename(columns={'chr1': 'chrom1', 'chr2': 'chrom2'})
    fithic[ct]['start1'] = ( fithic[ct]['fragmentMid1'] - res // 2 ).astype(int)
    fithic[ct]['end1'] = ( fithic[ct]['fragmentMid1'] + res // 2 ).astype(int)
    fithic[ct]['start2'] = ( fithic[ct]['fragmentMid2'] - res // 2 ).astype(int)
    fithic[ct]['end2'] = ( fithic[ct]['fragmentMid2'] + res // 2 ).astype(int)

    fithic[ct] = fithic[ct].loc[(fithic[ct]['chrom1'] != 'chrM') & 
                                (fithic[ct]['chrom2'] != 'chrM')]
    fithic[ct] = fithic[ct].loc[(fithic[ct]['chrom1'] != fithic[ct]['chrom2']) |
                                (fithic[ct]['start1'] != fithic[ct]['start2'])]
    print(ct, 'N =', fithic[ct].shape[0])

fithic[ct].head()

NeuN+
NeuN+ N = 916769
NeuN-
NeuN- N = 1501491
EN
EN N = 1666238
IN
IN N = 2047841


,chrom1,fragmentMid1,chrom2,fragmentMid2,q-value,start1,end1,start2,end2
0,chr1,50000,chr2,113550000,6.752831e-14,0,100000,113500000,113600000
1,chr1,50000,chr2,113650000,2.436446e-03,0,100000,113600000,113700000
2,chr1,50000,chr2,242146764,3.434134e-07,0,100000,242096764,242196764
3,chr1,50000,chr4,50000,2.729082e-31,0,100000,0,100000
4,chr1,50000,chr4,190150000,2.004531e-36,0,100000,190100000,190200000


# Overlap significant fithic interactions with neuronal dot anchors

In [42]:
# This section processes one cell type at a time
# Select cell type
ct = 'NeuN+'

In [43]:
# Read Supp.Table with neuronal dot annotation
dot_anch = pd.read_excel(neuronal_dot_anchor_path, na_values='-', skiprows=2)
dot_anch.head()

,chrom,start,end,genes
0,chr1,18630000,18640000,PAX7
1,chr1,24930000,24940000,RUNX3
2,chr1,44410000,44420000,RNF220
3,chr1,46480000,46490000,DMBX1
4,chr1,47230000,47240000,"TAL1,AL135960.1"


In [44]:
# Get pairs of neuronal dot anchors
dot_df_bins = utils.regs_to_bins(dot_anch, 100_000, mode='all', return_index=False)
pair_sites = bf.pair_by_distance(dot_df_bins, min_sep=200_000, 
                                 max_sep=int(1e10), suffixes=('1', '2'))
print('N =', pair_sites.shape[0])
pair_sites.head()

N = 1420


,chrom1,start1,end1,genes1,chrom2,start2,end2,genes2
0,chr9,800000,900000,"DMRT1,DMRT2,DMRT3",chr9,14300000,14400000,"NFIB,AL136366.1"
1,chr9,900000,1000000,"DMRT1,DMRT2,DMRT3",chr9,14300000,14400000,"NFIB,AL136366.1"
2,chr9,1000000,1100000,"DMRT1,DMRT2,DMRT3",chr9,14300000,14400000,"NFIB,AL136366.1"
3,chr9,800000,900000,"DMRT1,DMRT2,DMRT3",chr9,21900000,22000000,"CDKN2A,CDKN2A-DT,AL359922.1"
4,chr9,900000,1000000,"DMRT1,DMRT2,DMRT3",chr9,21900000,22000000,"CDKN2A,CDKN2A-DT,AL359922.1"


In [45]:
# Overlap pairs of neuronal dot anchors with significant fithic interactions
pair_sites_fh = pair_sites.merge(fithic[ct], how='inner', 
                                 on=['chrom1', 'start1', 'end1', 'chrom2', 'start2', 'end2'])
pair_sites_fh.head()

,chrom1,start1,end1,genes1,chrom2,start2,end2,genes2,fragmentMid1,fragmentMid2,q-value
0,chr9,900000,1000000,"DMRT1,DMRT2,DMRT3",chr9,14300000,14400000,"NFIB,AL136366.1",950000,14350000,6.191789e-04
1,chr9,1000000,1100000,"DMRT1,DMRT2,DMRT3",chr9,14300000,14400000,"NFIB,AL136366.1",1050000,14350000,2.372962e-06
2,chr9,800000,900000,"DMRT1,DMRT2,DMRT3",chr9,21900000,22000000,"CDKN2A,CDKN2A-DT,AL359922.1",850000,21950000,1.773365e-39
3,chr9,900000,1000000,"DMRT1,DMRT2,DMRT3",chr9,21900000,22000000,"CDKN2A,CDKN2A-DT,AL359922.1",950000,21950000,2.447484e-46
4,chr9,1000000,1100000,"DMRT1,DMRT2,DMRT3",chr9,21900000,22000000,"CDKN2A,CDKN2A-DT,AL359922.1",1050000,21950000,2.181005e-139


In [38]:
# Save result
if ct in ['NeuN+', 'EN', 'IN']:
    out_path = f'../data/neuronal_dots.{ct}.bedpe.gz'
elif ct == 'NeuN-':
    out_path = '../data/non-neuronal_dots.bedpe.gz'
else:
    raise ValueError()
    
pair_sites_fh.drop(columns=['fragmentMid1', 'fragmentMid2'])\
             .to_csv(out_path, header=True, index=False, sep='\t')